In [ ]:
# Determine if this is a benchmarking run
try:
    __benchmark__
except NameError:
    __benchmark__ = False

# Light travel time delay
A simple tutorial on how the finite speed of light affects exoplanet light curves and how to account for this in `starry`.

## Imports

In [ ]:
from starry import Star, Planet, System
import numpy as np
import matplotlib.pyplot as pl

In [ ]:
%matplotlib inline

## Instantiating a simple system

Let's instantiate a simple limb-darkened star and a hot Jupiter on a 10-day period with a dipole map:

In [ ]:
star = Star()
star.map[1] = 0.40
star.map[2] = 0.26
planet = Planet(L=1e-3, r=0.1, porb=10, prot=10, a=22)
planet.map[1, 0] = 0.5

By default, `starry` does not compute the effects of light travel time. The main reason for this is that all the stuff in `starry` is unitless: the star's physical properties (radius and luminosity) are fixed at unity, and the planet's parameters are all specified relative to these quantities. In order to compute the light travel time within the system, `starry` needs to know the actual size of the orbits. Since all distances are measured relative to the star's radius, to turn light travel effects on all you need to do is tell `starry` how big the star is. This is done by setting the `scale` parameter when instantiating a system:

In [ ]:
system = System([star, planet], scale=0)

I set it to zero for now, which tells `starry` the star's radius is **zero** (`scale` is measured in Solar units). Quite unphysical, but the `scale` parameter is **only** used to compute the light travel time, so this effectively tells `starry` that light travels across the system instantaneously. Let's plot the system light curve over one orbital period:

In [ ]:
time = np.concatenate((np.linspace(0, 0.5, 10000, endpoint=False),
                       np.linspace(0.5, 4.5, 100, endpoint=False),
                       np.linspace(4.5, 5.5, 10000, endpoint=False),
                       np.linspace(5.5, 9.5, 100, endpoint=False),
                       np.linspace(9.5, 10, 10000)))
phase = time / 10.
system.compute(time)
fig, ax = pl.subplots(1, figsize=(16, 4))
ax.plot(phase, system.flux);
ax.set_xlabel('Orbital phase', fontsize=16);
ax.set_ylabel('Flux', fontsize=16);
transit = phase[np.argmax(planet.z)]
eclipse = phase[np.argmin(planet.z)]
ax.axvline(transit, ls='--', lw=1);
ax.axvline(eclipse, ls='--', lw=1);
print("Transit: %.6f" % transit)
print("Eclipse: %.6f" % eclipse)

A couple remarks:

- I added higher time resolution near transit and secondary eclipse so we can time things accurately
- I computed and marked the phase corresponding to the largest value of `z` (when the planet is closest to the observer): this is the time of transit, accurate to within our time grid spacing
- I also computed and marked the phase corresponding to the smallest value of `z` (when the planet is farthest from the observer): this is the time of secondary eclipse

As expected, transit occurs at a phase of exactly 0 and eclipse at a phase of exactly 1/2.

In [ ]:
if __benchmark__:
    assert transit == 0
    assert eclipse == 0.5

## Accounting for the light travel time

Now, let's tell `starry` to compute the effects of light travel time by giving our star an actual physical radius. The `scale` parameter is in solar units, so if our star is a Sun-like star, we simply tell `starry`

In [ ]:
system.scale = 1

Let's recompute and plot the light curve:

In [ ]:
eps = 1e-3
time = np.concatenate((np.linspace(0, 0.5, 100000, endpoint=False),
                       np.linspace(0.5, 5 - eps, 100, endpoint=False),
                       np.linspace(5 - eps, 5 + eps, 1000000, endpoint=False),
                       np.linspace(5 + eps, 9.5, 100, endpoint=False),
                       np.linspace(9.5, 10, 100000)))
phase = time / planet.porb
system.compute(time)
fig, ax = pl.subplots(1, figsize=(16, 4))
ax.plot(phase, system.flux);
ax.set_xlabel('Orbital phase', fontsize=16);
ax.set_ylabel('Flux', fontsize=16);
transit = phase[np.argmin(np.abs(planet.x) + (planet.z < 0))]
eclipse = phase[np.argmin(np.abs(planet.x) + (planet.z > 0))]
ax.axvline(transit, ls='--', lw=1);
ax.axvline(eclipse, ls='--', lw=1);
print("Transit: %.6f" % transit)
print("Eclipse: %.6f" % eclipse)

In [ ]:
if __benchmark__:
    assert transit == 0.999940999409994
    assert eclipse == 0.5000590894

The effect is super tiny in this case, but there is now a small offset in both the time of transit and of secondary eclipse! In `starry`, the time delay is computed relative to the barycenter, so transits happen *early* (since the planet is closer to the observer than the barycenter) and eclipses happen *late* (since the planet is farther). We can verify that the phase difference between secondary eclipse and a phase of 0.5 is just `a / c`, the time light takes to travel from the planet to the barycenter:

In [ ]:
print("Time delay from starry (s): %.8f" % ((eclipse - 0.5) * planet.porb * 86400))

In [ ]:
c = 299792458. # m / s
RSUN = 6.957e8 # m
a = planet.a * system.scale * RSUN
print("Time delay from common sense (s): %.8f" % (a / c))

Not bad! Note that they are in fact **slightly** different, for a rather subtle reason. There's actually no closed-form way to compute the light travel time delay at arbitrary points in the light curve! Let me clarify: it's (very) easy to compute what the delay `dt` is given knowledge of where the planet is in the orbit at time `t`. But that calculation tells me what the light curve will look like at time `t + dt`, when the information actually reaches the observer (well, technically, when it reaches the reference point). Our observations are at time `t`, not `t + dt`, so what we actually need to know is where the planet was in its orbit at time `t - dt`. This may sound easy, but if you think about it, we are essentially asking to know the position

$x'(t) = x\bigg(t - dt\Big(x'(t)\Big)\bigg)$

which is a transcendental equation! The difficulty boils down to the fact that $dt$ is a function of the planet's retarded position, so it's a function of the very variable whose value we seek. 

To get around this, what we do in `starry` is we Taylor expand the equation above to second order, so we get a pretty good approximation to $dt(x'(t))$. We then re-compute the orbital position at the retarded time and use that to compute light curves. So, long story short, the time delay computed by `starry` is not exact. In principle, we could take the expansion out to higher order. If you have an application that needs this extra accuracy, please let us know!

## Just for fun

Finally, just for fun, here's a bunch of light curves with ridiculous (and very unphysical) light travel time delays, obtained by setting the stellar radius to very large (and very unphysical) values. I'm only plotting the *planet* flux so we can better see the behaviour of the phase curve.

In [ ]:
time = np.linspace(0, 10, 100000)
phase = time / planet.porb

fig, ax = pl.subplots(1, figsize=(16, 4))

for scale in [1, 10, 100, 1000, 1500]:
    system.scale = scale
    system.compute(time)
    ax.plot(phase, planet.flux, label="scale = %d" % scale);

ax.set_xlim(0.4, 0.7)
ax.set_xlabel('Orbital phase', fontsize=16);
ax.set_ylabel('Flux', fontsize=16);
ax.legend();

The cool thing you can see from these is that the light travel time also affects the phase curves, although keep in mind the effect is very small! Remember that these are unphysical systems: the purple curve corresponds to a planet traveling at $0.5c$ (!!) You'd probably have all sorts of relativistic effects in that case that `starry` does not model.

In [ ]:
if __benchmark__:
    # Run some extra tests here
    c = 299792458.
    RSUN = 6.957e8
    time = np.concatenate((np.linspace(0, 0.5, 100000, endpoint=False),
                           np.linspace(0.5, 4.5, 100, endpoint=False),
                           np.linspace(4.5, 5.5, 100000, endpoint=False),
                           np.linspace(5.5, 9.5, 100, endpoint=False),
                           np.linspace(9.5, 10, 100000)))
    phase = time / planet.porb
    system.scale = 1
    
    # Test this on an inclined orbit
    planet.inc = 30
    system.compute(time)
    transit = phase[np.argmax(planet.z)]
    eclipse = phase[np.argmin(planet.z)]
    delay_starry = (0.5 - (transit - eclipse)) * planet.porb * 86400
    a = planet.a * system.scale * RSUN
    delay_analytic = 2 * a / c * np.sin(planet.inc * np.pi / 180)
    assert np.abs(1 - delay_starry / delay_analytic) < 0.01
    
    # Test this on an orbit with nonzero Omega
    planet.inc = 90
    planet.Omega = 90
    system.compute(time)
    transit = phase[np.argmax(planet.z)]
    eclipse = phase[np.argmin(planet.z)]
    delay_starry = (0.5 - (transit - eclipse)) * planet.porb * 86400
    a = planet.a * system.scale * RSUN
    delay_analytic = 2 * a / c
    assert np.abs(1 - delay_starry / delay_analytic) < 0.01
    
    # Compute the light travel time from the secondary
    # eclipse to the barycenter for an eccentric orbit.
    # First, compute the phase of secondary eclipse
    # with no light travel time delay:
    planet.inc = 90
    planet.Omega = 0
    planet.w = 30
    planet.ecc = 0.25
    system.scale = 0
    system.compute(time)
    eclipse0 = phase[np.argmin(planet.z)]
    z0 = planet.z[np.argmin(planet.z)]
    # Now, compute the phase w/ the delay
    system.scale = 1
    system.compute(time)
    eclipse = phase[np.argmin(planet.z)]
    # Compute the light travel time to the barycenter in days:
    travel_time_starry = (eclipse - eclipse0) * planet.porb * 86400
    # Compute the analytic light travel time to the barycenter in days:
    travel_time_analytic = (0 - z0) * system.scale * RSUN / c
    assert np.abs(1 - travel_time_starry / travel_time_analytic) < 0.01